# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |

## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.
* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.
* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

You can find the best combinations of these parameters with [bench_speculative.py](https://github.com/sgl-project/sglang/blob/main/scripts/playground/bench_speculative.py).

In the documentation below, we set `--cuda-graph-max-bs` to be a small value for faster engine startup. For your own workloads, please tune the above parameters together with `--cuda-graph-max-bs`, `--max-running-requests`, `--mem-fraction-static` for the best performance. 

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

W0817 09:47:53.826000 4026298 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 09:47:53.826000 4026298 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 09:47:55] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=38998, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=31628

[2025-08-17 09:47:55] Using default HuggingFace chat template with detected content format: string


W0817 09:48:04.226000 4027077 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 09:48:04.226000 4027077 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0817 09:48:04.269000 4027076 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 09:48:04.269000 4027076 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 09:48:05] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-17 09:48:05] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=32.49 GB): 100%|██████████| 4/4 [00:09<00:00,  2.41s/it]
[2025-08-17 09:48:24] Capture draft cuda graph end. Time elapsed: 10.22 s. mem usage=0.26 GB. avail mem=32.47 GB.
[2025-08-17 09:48:24] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=32.47 GB


Capturing batches (bs=2 avail_mem=32.40 GB):  50%|█████     | 2/4 [00:00<00:00, 13.24it/s]

Capturing batches (bs=1 avail_mem=32.37 GB): 100%|██████████| 4/4 [00:00<00:00, 17.58it/s]
[2025-08-17 09:48:25] Capture draft extend cuda graph end. Time elapsed: 1.16 s. mem usage=0.10 GB. avail mem=32.37 GB.
[2025-08-17 09:48:25] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=32.37 GB
[2025-08-17 09:48:25] INFO:     Started server process [4026298]
[2025-08-17 09:48:25] INFO:     Waiting for application startup.
[2025-08-17 09:48:25] INFO:     Application startup complete.
[2025-08-17 09:48:25] INFO:     Uvicorn running on http://127.0.0.1:38998 (Press CTRL+C to quit)


[2025-08-17 09:48:26] INFO:     127.0.0.1:56972 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-17 09:48:26] INFO:     127.0.0.1:56984 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 09:48:26] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 09:48:26] INFO:     127.0.0.1:56996 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 09:48:26] The server is fired up and ready to roll!


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-17 09:48:31] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-17 09:48:31] INFO:     127.0.0.1:57010 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

W0817 09:48:38.298000 4030618 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 09:48:38.298000 4030618 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 09:48:39] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=37298, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=6680243

[2025-08-17 09:48:40] Using default HuggingFace chat template with detected content format: string


W0817 09:48:46.739000 4031049 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 09:48:46.739000 4031049 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0817 09:48:46.846000 4031050 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 09:48:46.846000 4031050 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 09:48:48] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-17 09:48:48] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=36.25 GB): 100%|██████████| 4/4 [00:16<00:00,  4.09s/it]
[2025-08-17 09:49:57] Capture draft cuda graph end. Time elapsed: 17.05 s. mem usage=-9.16 GB. avail mem=45.59 GB.
[2025-08-17 09:49:57] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=45.59 GB


Capturing batches (bs=1 avail_mem=4.36 GB): 100%|██████████| 4/4 [00:00<00:00, 28.12it/s]
[2025-08-17 09:49:58] Capture draft extend cuda graph end. Time elapsed: 0.92 s. mem usage=41.23 GB. avail mem=4.36 GB.
[2025-08-17 09:49:58] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=4.36 GB


[2025-08-17 09:49:58] INFO:     Started server process [4030618]
[2025-08-17 09:49:58] INFO:     Waiting for application startup.
[2025-08-17 09:49:58] INFO:     Application startup complete.
[2025-08-17 09:49:58] INFO:     Uvicorn running on http://127.0.0.1:37298 (Press CTRL+C to quit)
[2025-08-17 09:49:58] INFO:     127.0.0.1:46188 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-17 09:49:59] INFO:     127.0.0.1:46200 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 09:49:59] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 09:50:01] INFO:     127.0.0.1:46208 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 09:50:01] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-17 09:50:03] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-17 09:50:04] INFO:     127.0.0.1:46220 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

W0817 09:50:11.338000 4040464 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 09:50:11.338000 4040464 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 09:50:12] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=32651, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, rand

[2025-08-17 09:50:13] Using default HuggingFace chat template with detected content format: string


W0817 09:50:19.713000 4041108 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 09:50:19.713000 4041108 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0817 09:50:20.016000 4041107 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 09:50:20.016000 4041107 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


...

Capturing batches (bs=1 avail_mem=53.84 GB): 100%|██████████| 4/4 [00:00<00:00, 88.60it/s]
[2025-08-17 09:50:52] Capture draft extend cuda graph end. Time elapsed: 0.66 s. mem usage=0.22 GB. avail mem=53.84 GB.
[2025-08-17 09:50:52] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=8192, available_gpu_mem=53.84 GB


[2025-08-17 09:50:52] INFO:     Started server process [4040464]
[2025-08-17 09:50:52] INFO:     Waiting for application startup.
[2025-08-17 09:50:52] INFO:     Application startup complete.
[2025-08-17 09:50:52] INFO:     Uvicorn running on http://127.0.0.1:32651 (Press CTRL+C to quit)


[2025-08-17 09:50:53] INFO:     127.0.0.1:50610 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-17 09:50:53] INFO:     127.0.0.1:50624 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 09:50:53] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 09:50:54] INFO:     127.0.0.1:50630 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 09:50:54] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-17 09:50:58] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 09:50:58] INFO:     127.0.0.1:41932 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

W0817 09:51:05.614000 4044320 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 09:51:05.614000 4044320 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 09:51:07] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=34151, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_see

[2025-08-17 09:51:07] Using default HuggingFace chat template with detected content format: string


W0817 09:51:15.030000 4044827 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 09:51:15.030000 4044827 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0817 09:51:15.030000 4044828 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 09:51:15.030000 4044828 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 09:51:15] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=58.24 GB): 100%|██████████| 4/4 [00:05<00:00,  1.29s/it]
[2025-08-17 09:51:43] Capture draft cuda graph end. Time elapsed: 5.66 s. mem usage=-0.14 GB. avail mem=58.20 GB.
[2025-08-17 09:51:43] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=58.20 GB


Capturing batches (bs=1 avail_mem=58.08 GB): 100%|██████████| 4/4 [00:00<00:00, 24.17it/s]
[2025-08-17 09:51:44] Capture draft extend cuda graph end. Time elapsed: 0.98 s. mem usage=0.11 GB. avail mem=58.08 GB.
[2025-08-17 09:51:44] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=58.08 GB


[2025-08-17 09:51:44] INFO:     Started server process [4044320]
[2025-08-17 09:51:44] INFO:     Waiting for application startup.
[2025-08-17 09:51:44] INFO:     Application startup complete.
[2025-08-17 09:51:44] INFO:     Uvicorn running on http://127.0.0.1:34151 (Press CTRL+C to quit)


[2025-08-17 09:51:45] INFO:     127.0.0.1:37282 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-17 09:51:45] INFO:     127.0.0.1:37284 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 09:51:45] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-17 09:51:46] INFO:     127.0.0.1:37286 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 09:51:46] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-17 09:51:50] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-17 09:51:50] INFO:     127.0.0.1:37288 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

W0817 09:51:58.938000 4048852 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 09:51:58.938000 4048852 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 09:52:00] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=31760, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=99309395, constrained_json

[2025-08-17 09:52:01] Using default HuggingFace chat template with detected content format: string


W0817 09:52:07.832000 4049793 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 09:52:07.832000 4049793 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0817 09:52:07.966000 4049794 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 09:52:07.966000 4049794 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 09:52:09] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-17 09:52:09] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=44.81 GB):  50%|█████     | 2/4 [00:01<00:01,  1.57it/s]

Capturing batches (bs=1 avail_mem=44.81 GB): 100%|██████████| 4/4 [00:01<00:00,  2.09it/s]
[2025-08-17 09:52:21] Capture draft cuda graph end. Time elapsed: 2.53 s. mem usage=2.61 GB. avail mem=42.20 GB.
[2025-08-17 09:52:21] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=42.16 GB


Capturing batches (bs=1 avail_mem=41.39 GB): 100%|██████████| 4/4 [00:00<00:00, 39.23it/s]
[2025-08-17 09:52:22] Capture draft extend cuda graph end. Time elapsed: 1.04 s. mem usage=0.78 GB. avail mem=41.39 GB.
[2025-08-17 09:52:22] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=32768, available_gpu_mem=41.39 GB


[2025-08-17 09:52:23] INFO:     Started server process [4048852]
[2025-08-17 09:52:23] INFO:     Waiting for application startup.
[2025-08-17 09:52:23] INFO:     Application startup complete.
[2025-08-17 09:52:23] INFO:     Uvicorn running on http://0.0.0.0:31760 (Press CTRL+C to quit)
[2025-08-17 09:52:23] INFO:     127.0.0.1:38336 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-17 09:52:24] INFO:     127.0.0.1:38346 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 09:52:24] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-17 09:52:24] INFO:     127.0.0.1:38356 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 09:52:24] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-08-17 09:52:28] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 09:52:28] Decode batch. #running-req: 1, #token: 87, token usage: 0.00, accept len: 1.67, cuda graph: True, gen throughput (token/s): 11.41, #queue-req: 0, 


[2025-08-17 09:52:29] Decode batch. #running-req: 1, #token: 154, token usage: 0.01, accept len: 1.68, cuda graph: True, gen throughput (token/s): 144.62, #queue-req: 0, 


[2025-08-17 09:52:29] Decode batch. #running-req: 1, #token: 220, token usage: 0.01, accept len: 1.65, cuda graph: True, gen throughput (token/s): 148.39, #queue-req: 0, 


[2025-08-17 09:52:29] Decode batch. #running-req: 1, #token: 285, token usage: 0.01, accept len: 1.62, cuda graph: True, gen throughput (token/s): 151.48, #queue-req: 0, 


...

[2025-08-17 09:52:30] Decode batch. #running-req: 1, #token: 419, token usage: 0.02, accept len: 1.68, cuda graph: True, gen throughput (token/s): 150.90, #queue-req: 0, 


[2025-08-17 09:52:31] Decode batch. #running-req: 1, #token: 490, token usage: 0.02, accept len: 1.77, cuda graph: True, gen throughput (token/s): 157.63, #queue-req: 0, 


[2025-08-17 09:52:31] Decode batch. #running-req: 1, #token: 552, token usage: 0.03, accept len: 1.55, cuda graph: True, gen throughput (token/s): 134.81, #queue-req: 0, 


[2025-08-17 09:52:32] Decode batch. #running-req: 1, #token: 619, token usage: 0.03, accept len: 1.68, cuda graph: True, gen throughput (token/s): 150.77, #queue-req: 0, 


[2025-08-17 09:52:32] Decode batch. #running-req: 1, #token: 684, token usage: 0.03, accept len: 1.62, cuda graph: True, gen throughput (token/s): 152.02, #queue-req: 0, 
[2025-08-17 09:52:32] INFO:     127.0.0.1:56690 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).